<a href="https://colab.research.google.com/github/x1001000/raspberrypi3-yamnet-sed/blob/main/colab_notebooks/%E5%8F%B0%E7%81%A3%E7%9A%84%E6%95%91%E8%AD%B7%E8%BB%8A%E3%80%81%E8%AD%A6%E8%BB%8A%E3%80%81%E6%B6%88%E9%98%B2%E8%BB%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# At runtime reset

## install

In [ ]:
!pip install soundfile
!pip install git+https://github.com/nficano/pytube
!pip install pydub

  Cloning https://github.com/nficano/pytube to /tmp/pip-req-build-x31gc2yf
  Running command git clone -q https://github.com/nficano/pytube /tmp/pip-req-build-x31gc2yf
  Created wheel for pytube: filename=pytube-10.5.3-cp37-none-any.whl size=42794 sha256=84bc8761e4b53337304c068708b0ca46979de9ea3d914f5a3d9f3813155b86e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-eq0ylop8/wheels/44/da/40/3b5e03abe33a91895343814fb44b309512375408f4a909555b
Successfully built pytube


## download YAMNet (15M bytes)

In [ ]:
# !curl -O https://storage.googleapis.com/audioset/yamnet.h5
# !git clone https://github.com/tensorflow/models
# !cp models/research/audioset/yamnet/* .
!git clone https://github.com/x1001000/raspberrypi3-yamnet-sed
!cp raspberrypi3-yamnet-sed/yamnet/* .

Cloning into 'raspberrypi3-yamnet-sed'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 285 (delta 9), reused 9 (delta 3), pack-reused 262
Receiving objects: 100% (285/285), 32.96 MiB | 25.15 MiB/s, done.
Resolving deltas: 100% (161/161), done.


# At runtime restart

## import

In [ ]:
import numpy as np
import resampy
import soundfile as sf
import tensorflow as tf

import params as yamnet_params
import yamnet as yamnet_model

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

from pytube import Playlist, YouTube
from pydub import AudioSegment

from IPython.display import display, Audio
from time import sleep

## load YAMNet (3.7M params)

In [ ]:
params = yamnet_params.Params()
yamnet = yamnet_model.yamnet_frames_model(params)
yamnet.load_weights('yamnet.h5')
yamnet_classes = yamnet_model.class_names('yamnet_class_map_zh-tw.csv')
yamnet_classes = np.concatenate([yamnet_classes, np.array(['台灣垃圾車'])])

#yamnet.summary()

## def data_from_YouTube(url)

In [ ]:
def data_from_YouTube(url):
    title = YouTube(url).title
    stream = YouTube(url).streams.get_audio_only()
    if stream:
        default_filename = stream.default_filename
        print(stream.download(), '✅已下載，匯出wav檔...', end='')
        AudioSegment.from_file(default_filename).export(default_filename+'.wav', format='wav')
        print('匯出waveform陣列...')
        file_name = default_filename+'.wav'

        # https://github.com/tensorflow/models/blob/master/research/audioset/yamnet/inference.py#L40
        wav_data, sr = sf.read(file_name, dtype=np.int16)
        assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
        waveform = wav_data / tf.int16.max#32768.0  # Convert to [-1.0, +1.0]
        waveform = waveform.astype('float32')

        # Convert to mono and the sample rate expected by YAMNet.
        if len(waveform.shape) > 1:
            waveform = np.mean(waveform, axis=1)
        if sr != params.sample_rate:
            waveform = resampy.resample(waveform, sr, params.sample_rate)
        
        return title, waveform
    else:
        print(f'No audio_only stream from {title}!')

## def data_from_Playlist(url)

In [ ]:
def data_from_Playlist(url, begin=1, end=None):
    titles = []
    waveforms = []
    for url in Playlist(url).video_urls[begin-1:end]:
        title, waveform = data_from_YouTube(url)
        waveforms.append(waveform)
        titles.append(title)
        # sleep(10) # HTTP 429 hack
    return titles, waveforms

# Run inference on a playlist

## 救護車

In [ ]:
playlist = ['https://www.youtube.com/playlist?list=PL0Q2eQA7p-wSQp7_JiKgEuQSwJiiwvq7M'] # 救護車

MA = 5
SR = int(params.sample_rate)

titles, waveforms = data_from_Playlist(*playlist)
print('\nRunning inference...')
for title, waveform in zip(titles, waveforms):
    print('\n'+title)
    # display(Audio(waveform, rate=SR)) # crashes if waveform is big
    for i in range(MA, len(waveform)//SR, MA):
        scores, embeddings, spectrogram = yamnet(waveform[(i-MA)*SR:i*SR])
        prediction = np.mean(scores[:-1], axis=0)
        top5 = np.argsort(prediction)[::-1][:5]
        print(f'{i//60}:{i%60:2d}',
            ''.join(f" {prediction[i]:.2f} 👉{yamnet_classes[i][:12].ljust(12, '　')}" for i in top5))

/content/台灣救護車音效.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...
/content/台北市救護車緊急出動 Taipei Ambulance Responding.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...
/content/Mercedes-Benz AMBULANCE 賓士救護車出勤.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...
/content/大台北民間救護車緊急出動 Private Ambulances Responding in Metro Taipei.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...

Running inference...

台灣救護車音效
0: 5  0.53 👉應急車輛　　　　　　　　 0.46 👉車輛　　　　　　　　　　 0.44 👉警笛　　　　　　　　　　 0.41 👉救護車（警報）　　　　　 0.40 👉機動車輛（道路）　　　　
0:10  0.42 👉應急車輛　　　　　　　　 0.36 👉機動車輛（道路）　　　　 0.35 👉救護車（警報）　　　　　 0.34 👉車輛　　　　　　　　　　 0.34 👉警笛　　　　　　　　　　
0:15  0.47 👉應急車輛　　　　　　　　 0.43 👉警笛　　　　　　　　　　 0.38 👉救護車（警報）　　　　　 0.36 👉車輛　　　　　　　　　　 0.35 👉機動車輛（道路）　　　　
0:20  0.57 👉應急車輛　　　　　　　　 0.50 👉救護車（警報）　　　　　 0.50 👉警笛　　　　　　　　　　 0.39 👉車輛　　　　　　　　　　 0.39 👉報警　　　　　　　　　　
0:25  0.64 👉應急車輛　　　　　　　　 0.60 👉警笛　　　　　　　　　　 0.55 👉救護車（警報）　　　　　 0.50 👉報警　　　　　　　　　　 0.45 👉機動車輛（道路）　　　　
0:30  0.66 👉應急車輛　　　　　　　　 0.58 👉警笛　　　　　　　　　　 0.53 👉救護車（警報）　　　　　 0.48 👉機動車輛（道路）　　　　 0.46 👉報警　　　　　　　　　　
0:35  0.72 👉應急車輛　　　　　　　　 0.66 👉警笛　　　　　　　　　　 0.59 👉救護車

## 警車

In [ ]:
playlist = ['https://www.youtube.com/playlist?list=PL0Q2eQA7p-wQkx2hcJMyD-7fB6xqhRz_B'] # 警車

MA = 5
SR = int(params.sample_rate)

titles, waveforms = data_from_Playlist(*playlist)
print('\nRunning inference...')
for title, waveform in zip(titles, waveforms):
    print('\n'+title)
    # display(Audio(waveform, rate=SR)) # crashes if waveform is big
    for i in range(MA, len(waveform)//SR, MA):
        scores, embeddings, spectrogram = yamnet(waveform[(i-MA)*SR:i*SR])
        prediction = np.mean(scores[:-1], axis=0)
        top5 = np.argsort(prediction)[::-1][:5]
        print(f'{i//60}:{i%60:2d}',
            ''.join(f" {prediction[i]:.2f} 👉{yamnet_classes[i][:12].ljust(12, '　')}" for i in top5))

/content/警車警笛聲 EP1 提供大家下載！！！.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...
/content/警車警笛聲 EP2 標準警笛！！.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...
/content/警車警笛聲 EP3 臺灣警車警笛分類.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...
/content/臺灣警車鈴聲(緊急處理用)完整版.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...

Running inference...

警車警笛聲 EP.1 提供大家下載！！！
0: 5  0.97 👉警笛　　　　　　　　　　 0.96 👉報警　　　　　　　　　　 0.35 👉警車（警報）　　　　　　 0.20 👉應急車輛　　　　　　　　 0.14 👉救護車（警報）　　　　　
0:10  0.97 👉警笛　　　　　　　　　　 0.96 👉報警　　　　　　　　　　 0.41 👉警車（警報）　　　　　　 0.25 👉應急車輛　　　　　　　　 0.16 👉救護車（警報）　　　　　
0:15  0.97 👉警笛　　　　　　　　　　 0.96 👉報警　　　　　　　　　　 0.37 👉警車（警報）　　　　　　 0.20 👉應急車輛　　　　　　　　 0.15 👉救護車（警報）　　　　　
0:20  0.97 👉警笛　　　　　　　　　　 0.96 👉報警　　　　　　　　　　 0.33 👉警車（警報）　　　　　　 0.21 👉應急車輛　　　　　　　　 0.13 👉救護車（警報）　　　　　
0:25  0.95 👉警笛　　　　　　　　　　 0.93 👉報警　　　　　　　　　　 0.46 👉警車（警報）　　　　　　 0.26 👉應急車輛　　　　　　　　 0.18 👉救護車（警報）　　　　　
0:30  0.96 👉警笛　　　　　　　　　　 0.95 👉報警　　　　　　　　　　 0.40 👉警車（警報）　　　　　　 0.27 👉應急車輛　　　　　　　　 0.17 👉救護車（警報）　　　　　
0:35  0.96 👉警笛　　　　　　　　　　 0.95 👉報警　　　　　　　　　　 0.42 👉警車（警報）　　　　　　 0.25 👉應急車輛　　　　　　　　 0.17 👉救護車（警報）　　　　　

警車警

## 消防車

In [ ]:
playlist = ['https://www.youtube.com/playlist?list=PL0Q2eQA7p-wQ29QPFe6LfvWRMqId-N_75'] # 消防車

MA = 5
SR = int(params.sample_rate)

titles, waveforms = data_from_Playlist(*playlist)
print('\nRunning inference...')
for title, waveform in zip(titles, waveforms):
    print('\n'+title)
    # display(Audio(waveform, rate=SR)) # crashes if waveform is big
    for i in range(MA, len(waveform)//SR, MA):
        scores, embeddings, spectrogram = yamnet(waveform[(i-MA)*SR:i*SR])
        prediction = np.mean(scores[:-1], axis=0)
        top5 = np.argsort(prediction)[::-1][:5]
        print(f'{i//60}:{i%60:2d}',
            ''.join(f" {prediction[i]:.2f} 👉{yamnet_classes[i][:12].ljust(12, '　')}" for i in top5))

/content/消防車的聲音.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...
/content/20190719 協助大昌分隊消防車開道.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...
/content/消防車出勤鳴笛的震撼.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...
/content/台南市消防車緊急出動 Tainan City Fire Engine Response Demo.mp4 ✅已下載，匯出wav檔...匯出waveform陣列...

Running inference...

消防車的聲音
0: 5  0.71 👉警車（警報）　　　　　　 0.68 👉警笛　　　　　　　　　　 0.64 👉報警　　　　　　　　　　 0.62 👉應急車輛　　　　　　　　 0.41 👉機動車輛（道路）　　　　
0:10  0.82 👉警車（警報）　　　　　　 0.73 👉警笛　　　　　　　　　　 0.71 👉報警　　　　　　　　　　 0.68 👉應急車輛　　　　　　　　 0.44 👉機動車輛（道路）　　　　
0:15  0.74 👉應急車輛　　　　　　　　 0.72 👉警車（警報）　　　　　　 0.65 👉警笛　　　　　　　　　　 0.61 👉報警　　　　　　　　　　 0.56 👉車輛　　　　　　　　　　
0:20  0.71 👉警車（警報）　　　　　　 0.70 👉應急車輛　　　　　　　　 0.68 👉警笛　　　　　　　　　　 0.65 👉報警　　　　　　　　　　 0.46 👉機動車輛（道路）　　　　
0:25  0.80 👉警車（警報）　　　　　　 0.73 👉警笛　　　　　　　　　　 0.71 👉應急車輛　　　　　　　　 0.71 👉報警　　　　　　　　　　 0.48 👉機動車輛（道路）　　　　
0:30  0.70 👉應急車輛　　　　　　　　 0.70 👉警車（警報）　　　　　　 0.67 👉警笛　　　　　　　　　　 0.64 👉報警　　　　　　　　　　 0.48 👉車輛　　　　　　　　　　

20190719 協助大昌分隊消防車開道
0: 5  0.61 👉車輛　　　　　　　　　　 0.56 👉應急車輛　　　　　　　　 0.43 👉消防車，消防車（警笛）　 0.42 👉機動車輛（道路）　　　　 